In [ ]:
# %%
# rebinding_and_damage_pipeline.ipynb - Annotated Notebook Version
# Cell 1: Imports and Configuration
import numpy as np                           # Numerical operations
import xarray as xr                          # Multi-dimensional labeled arrays
import matplotlib.pyplot as plt              # Plotting
import cartopy.crs as ccrs                   # Map projections
import pandas as pd                          # Table and DataFrame operations

In [ ]:
import pandas as pd
import xarray as xr

dir_list_df = pd.read_csv('/user/ab5405/summeraliaclimate/code/dir_list.csv', index_col=0)
dir_list = dir_list_df['dir_path'].to_dict()


In [ ]:
from funcs_support import get_filepaths

df_paths = get_filepaths(source_dir='proc', dir_list=dir_list)
df_paths.head()

In [ ]:
# %%
# Cell 2: 10°F Rebin Function Definition

def rebin_to_10F_by_groupby(da):
    """
    Re-bin a DataArray from fine Kelvin bins ('bin_f') into 10°F bins.
    Returns dims ('bin10F','lat','lon') with coords:
      - bin10F    : integer 0..9
      - mid_F_degF: midpoint temperature in °F for each bin
    """
    # (a) Define finite 0–100°F edges
    edges_F = np.concatenate(([0], np.arange(10, 100, 10), [100]))
    # (b) Convert °F edges to Kelvin
    edges_K = (edges_F + 459.67) * (5/9)
    # (c) Compute midpoints in °F
    mid_F = (edges_F[:-1] + edges_F[1:]) / 2

    # (d) Extract fine-bin centers (in K)
    fine_vals = da['bin_f'].values
    # (e) Digitize into 10 groups (0–9)
    indices = np.digitize(fine_vals, edges_K) - 1
    # (f) Clip out-of-range
    indices = np.clip(indices, 0, len(mid_F) - 1)

    # (g) Assign integer group labels
    da2 = da.assign_coords(bin10F=('bin_f', indices))
    # (h) Sum to collapse fine bins into coarse bins
    rebinned = da2.groupby('bin10F').sum(dim='bin_f')

    # (i) Attach coords: integer labels & real midpoints
    rebinned = rebinned.assign_coords({
        'bin10F': np.arange(rebinned.sizes['bin10F']),
        'mid_F_degF': ('bin10F', mid_F)
    })
    return rebinned


In [ ]:
df = df_paths.query("varname == 'tasdmgfparams' and gwl == 'ALLGWLs'").copy()

In [ ]:
dmgf = xr.DataArray([0.69,0.59,0.64,0.36,0.27,0,0.12,0.23,0.33,0.94],dims=['bin_rebinned'],coords=[np.arange(1,11)])
bins_dng = np.append(np.append(-np.inf,(np.arange(10,91,10)+459.67)*5/9),np.inf)
bin_name_dng = '_dng'

In [ ]:
print(dmgf.dims)
dmgf_10F = dmgf.rename({'bin_rebinned':'bin10F'})
print(dmgf_10F.dims)

In [ ]:
# %%
# Cell 2: 10°F Rebin Function Definition

def rebin_to_10F_by_groupby(da):
    """
    Re-bin a DataArray from fine Kelvin bins ('bin_f') into 10°F bins.
    Returns dims ('bin10F','lat','lon') with coords:
      - bin10F    : integer 0..9
      - mid_F_degF: midpoint temperature in °F for each bin
    """
    # (a) Define finite 0–100°F edges
    edges_F = np.concatenate(([0], np.arange(10, 100, 10), [100]))
    # (b) Convert °F edges to Kelvin
    edges_K = (edges_F + 459.67) * (5/9)
    # (c) Compute midpoints in °F
    mid_F = (edges_F[:-1] + edges_F[1:]) / 2

    # (d) Extract fine-bin centers (in K)
    fine_vals = da['bin_f'].values
    # (e) Digitize into 10 groups (0–9)
    indices = np.digitize(fine_vals, edges_K) - 1
    # (f) Clip out-of-range
    indices = np.clip(indices, 0, len(mid_F) - 1)

    # (g) Assign integer group labels
    da2 = da.assign_coords(bin10F=('bin_f', indices))
    # (h) Sum to collapse fine bins into coarse bins
    rebinned = da2.groupby('bin10F').sum(dim='bin_f')

    # (i) Attach coords: integer labels & real midpoints
    rebinned = rebinned.assign_coords({
        'bin10F': np.arange(rebinned.sizes['bin10F']),
        'mid_F_degF': ('bin10F', mid_F)
    })
    return rebinned


In [ ]:
# Cell 3: 3°C Rebin Function Definition

def rebin_to_3C_by_groupby(da):
    """
    Re-bin a DataArray from fine Kelvin bins ('bin_f') into 3°C bins.
    Returns dims ('bin3C','lat','lon') with coords:
      - bin3C     : integer 0..N-1
      - mid_C_degC: midpoint temperature in °C for each bin
    """
    # (a) Define 3°C edges in °C, convert to Kelvin
    edges_C = np.concatenate(([-np.inf], np.arange(-15, 31, 3), [np.inf]))
    edges_K = edges_C + 273.15
    # (b) Compute midpoints in °C
    mid_C = (edges_C[:-1] + edges_C[1:]) / 2

    # (c) Digitize fine-bin centers into 3°C bins
    fine_vals = da['bin_f'].values
    indices = np.digitize(fine_vals, edges_K) - 1
    indices = np.clip(indices, 0, len(mid_C) - 1)

    # (d) Label groups
    da2 = da.assign_coords(bin3C=('bin_f', indices))
    # (e) Sum to collapse
    rebinned = da2.groupby('bin3C').sum(dim='bin_f')

    # (f) Attach integer labels & midpoints
    rebinned = rebinned.assign_coords({
        'bin3C': np.arange(rebinned.sizes['bin3C']),
        'mid_C_degC': ('bin3C', mid_C)
    })
    return rebinned


In [ ]:
# Cell 4: Define Damage Functions with Coordinates
# (a) Mortality (10°F bins)
mort_coeffs = [0.69, 0.59, 0.64, 0.36, 0.27, 0.00, 0.12, 0.23, 0.33, 0.94]
edges_F = np.arange(0, 110, 10)
mid_F = (edges_F[:-1] + edges_F[1:]) / 2

dmgf_10F = xr.DataArray(
    mort_coeffs,
    dims=['bin10F'],
    coords={
        'bin10F': np.arange(len(mort_coeffs)),
        'mid_F_degF': ('bin10F', mid_F)
    },
    name='mortality_coeff'
)


In [ ]:
# (b) GDP-loss (3°C bins)
gdp_coeffs = [0.01, 0.02, 0.03, 0.025, 0.02, 0.015, 0.01, 0.005, 0.002, 0.001, 0.0, -0.001, -0.002, -0.003, -0.004, -0.005]
edges_C = np.concatenate(([-np.inf], np.arange(-15, 31, 3), [np.inf]))
mid_C = (edges_C[:-1] + edges_C[1:]) / 2

dmgf_dnh_3C = xr.DataArray(
    gdp_coeffs,
    dims=['bin3C'],
    coords={
        'bin3C': np.arange(len(gdp_coeffs)),
        'mid_C_degC': ('bin3C', mid_C)
    },
    name='gdp_loss_coeff'
)

In [ ]:
# Cell 5: Load Dataset List and Compute Impacts
# Load your DataFrame of model paths (assumes df exists)
# df = pd.read_csv('dir_list.csv')

impact_dng_list = []
model_labels = []
for idx, row in df.iterrows():
    path = row['path']
    ds = xr.open_zarr(path)

    # select slices and ensure dims order
    old_g2 = ds['bins_behrer'].sel(gwl=2.0, method='nearest').transpose('bin_f','lat','lon')
    old_g06 = ds['bins_behrer'].sel(gwl=0.61, method='nearest').transpose('bin_f','lat','lon')

    # rebin to coarse bins
    reb2 = rebin_to_10F_by_groupby(old_g2)
    reb06 = rebin_to_10F_by_groupby(old_g06)
    diff_10F = reb2 - reb06

    # apply damage function
    mort_map = (diff_10F * dmgf_10F).sum(dim='bin10F')

    impact_dng_list.append(mort_map)
    model_labels.append(row['model'])

In [ ]:
# Cell 6: Concatenate Results and Plot
impact_all = xr.concat(impact_dng_list, dim='model')
impact_all = impact_all.assign_coords(model=model_labels)
ensemble_mean = impact_all.mean(dim='model')


In [ ]:

# plotting function definition
def plot_map(data, title, cbar_label):
    fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
    mesh = ax.pcolormesh(data['lon'], data['lat'], data.values,
                         transform=ccrs.PlateCarree(), cmap='Reds')
    cbar = fig.colorbar(mesh, ax=ax, orientation='vertical')
    cbar.set_label(cbar_label)
    ax.coastlines()
    ax.set_title(title)
    plt.show()


In [ ]:
# plot ensemble mean
plot_map(ensemble_mean,
         "Ensemble Mean Mortality Impact (2°C vs 0.61°C)",
         "Deaths per 100k")
